# Automated Project: Planning, Estimation, and Allocation

## Initial Imports

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [3]:
from helper import get_openai_api_key
#get_openai_api_key() # check if your key is loaded correctly

## Set OpenAI Model

In [4]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [5]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Pydantic Models for Structured Output

In [6]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    start_week: int = Field(..., description="Week number when the task starts (e.g., 1)")
    finish_week: int = Field(..., description="Week number when the task ends (e.g., 6)")
    estimated_time_days: float = Field(..., description="Estimated time to complete the task in days")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")
    deliverables: List[str] = Field(..., description="List of deliverables associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks

In [7]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan 
)

crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

## Crew's Inputs

In [8]:
from IPython.display import display, Markdown

project = 'Electric Motor Production Facility'
industry = 'Aerospace / Electric Motor Manufacturing'
project_objectives = """Design, develop, and launch a state-of-the-art production facility 
capable of manufacturing high-performance electric motors 
tailored for heavy-lift drone applications. 
The facility must support scalable, high-efficiency manufacturing 
and strict quality assurance protocols for aerospace-grade products."""
team_members = """
- Michael Schneider (Project Manager): Oversees project timeline, budget, and cross-functional coordination;
- Marina Volkova (Automation Engineer): Develops software and control systems for manufacturing automation;
- Kwame Mensah (Mechanical Designer): Designs motor components and production line equipment;
- Emily Carter (QA Engineer): Ensures motors meet aerospace-grade quality and reliability standards;
- Luca Romano (Manufacturing QA Engineer): Implements inspection processes and production audits;
"""
project_requirements = """
- Secure and prepare industrial-grade site with proper electrical, ventilation, and safety systems;
- Design an efficient production layout optimized for electric motor assembly, testing, and packaging;
- Install CNC machines, robotic arms, winding machines, 3D printing machines, precision assembly tools, and automated testing stations;
- Develop a digital Manufacturing Execution System (MES) for tracking production, inventory, and QC;
- Ensure compliance with aerospace manufacturing regulations (AS9100, ISO 9001);
- Create rigorous testing protocols for thermal, vibration, and torque performance validation;
- Implement cleanroom protocols or dust-control measures where required;
- Train staff on advanced assembly techniques, robotic operation, and safety procedures;
- Establish a supply chain for high-grade materials (magnets, copper wire, casings, etc.);
- Plan for future scalability (modular production lines, automation upgrades, robotic cell expansion, etc.);
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Electric Motor Production Facility

**Project Objectives:** Design, develop, and launch a state-of-the-art production facility 
capable of manufacturing high-performance electric motors 
tailored for heavy-lift drone applications. 
The facility must support scalable, high-efficiency manufacturing 
and strict quality assurance protocols for aerospace-grade products.

**Industry:** Aerospace / Electric Motor Manufacturing

**Team Members:**

- Michael Schneider (Project Manager): Oversees project timeline, budget, and cross-functional coordination;
- Marina Volkova (Automation Engineer): Develops software and control systems for manufacturing automation;
- Kwame Mensah (Mechanical Designer): Designs motor components and production line equipment;
- Emily Carter (QA Engineer): Ensures motors meet aerospace-grade quality and reliability standards;
- Luca Romano (Manufacturing QA Engineer): Implements inspection processes and production audits;

**Project Requirements:**

- Secure and prepare industrial-grade site with proper electrical, ventilation, and safety systems;
- Design an efficient production layout optimized for electric motor assembly, testing, and packaging;
- Install CNC machines, robotic arms, winding machines, 3D printing machines, precision assembly tools, and automated testing stations;
- Develop a digital Manufacturing Execution System (MES) for tracking production, inventory, and QC;
- Ensure compliance with aerospace manufacturing regulations (AS9100, ISO 9001);
- Create rigorous testing protocols for thermal, vibration, and torque performance validation;
- Implement cleanroom protocols or dust-control measures where required;
- Train staff on advanced assembly techniques, robotic operation, and safety procedures;
- Establish a supply chain for high-grade materials (magnets, copper wire, casings, etc.);
- Plan for future scalability (modular production lines, automation upgrades, robotic cell expansion, etc.);



## Kicking off the crew

In [9]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 323c78a3-137c-4b3e-8813-64d4f1e2ab49                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Task: Carefully analyze the project_requirements for the Electric Motor Production Facility project and break  │
│  them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure      │
│  that all dependencies are accounted for:                                                                       │
│                                                                                                                 │
│  - Secure and prepare industrial-grade site with proper electrical, ventilation, and safety systems;            │
│  - Design an efficient production layout optimized for electric motor assembly, testing, and packaging;         │
│  - Install CNC machines, robotic arms, winding machines, 3D printing machines, precision assembly tools, and    │
│  automated testing stations;                                                                                    │
│  - Develop a digital Manufacturing Execution System (MES) for tracking production, inventory, and QC;           │
│  - Ensure compliance with aerospace manufacturing regulations (AS9100, ISO 9001);                               │
│  - Create rigorous testing protocols for thermal, vibration, and torque performance validation;                 │
│  - Implement cleanroom protocols or dust-control measures where required;                                       │
│  - Train staff on advanced assembly techniques, robotic operation, and safety procedures;                       │
│  - Establish a supply chain for high-grade materials (magnets, copper wire, casings, etc.);                     │
│  - Plan for future scalability (modular production lines, automation upgrades, robotic cell expansion, etc.);   │
│                                                                                                                 │
│                                                                                                                 │
│  Team members:                                                                                                  │
│                                                                                                                 │
│  - Michael Schneider (Project Manager): Oversees project timeline, budget, and cross-functional coordination;   │
│  - Marina Volkova (Automation Engineer): Develops software and control systems for manufacturing automation;    │
│  - Kwame Mensah (Mechanical Designer): Designs motor components and production line equipment;                  │
│  - Emily Carter (QA Engineer): Ensures motors meet aerospace-grade quality and reliability standards;           │
│  - Luca Romano (Manufacturing QA Engineer): Implements inspection processes and production audits;              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Project Breakdown for Electric Motor Production Facility**                                                   │
│                                                                                                                 │
│  1. **Secure and Prepare Industrial-Grade Site**                                                                │
│     - **Scope:** Identify and secure a location suitable for manufacturing with the necessary infrastructure.   │
│  Acquire permits and ensure all essential utilities (electricity, water, etc.) are available.                   │
│     - **Tasks:**                                                                                                │
│        - Site selection and acquisition (2 weeks)                                                               │
│        - Utility assessments and contracts (1 week)                                                             │
│        - Environmental impact assessments (2 weeks)                                                             │
│        - Begin site preparations including earthworks and foundation checks (4 weeks)                           │
│     - **Timeline:** Weeks 1-9                                                                                   │
│     - **Dependencies:** None                                                                                    │
│     - **Deliverables:** Secured site and utility readiness.                                                     │
│                                                                                                                 │
│  2. **Design an Efficient Production Layout**                                                                   │
│     - **Scope:** Develop a comprehensive layout for the production facility that maximizes efficiency,          │
│  incorporates workflow, and adheres to safety standards.                                                        │
│     - **Tasks:**                                                                                                │
│        - Develop initial layout drafts (3 weeks)                                                                │
│        - Review and finalize production layout with input from all team leads (2 weeks)                         │
│        - Submit for approval and modifications based on feedback (1 week)                                       │
│        - Finalize mechanical designs for workstations (4 weeks)                                                 │
│     - **Timeline:** Weeks 10-20                                                                                 │
│     - **Dependencies:** Completion of site preparation                                                          │
│     - **Deliverables:** Finalized production layout design.                                                     │
│                                                                                                                 │
│  3. **Install CNC Machines and Automation Equipment**                                                           │
│     - **Scope:** Procure and install the required machinery including CNC machines, robotic arms, and assembly  │
│  tools.                                                                                                         │
│     - **Tasks:**                                       

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f7f58585-8217-4a9a-9a25-5cd4e9d20fd3                                                                     │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Task: Strategically allocate tasks for the Electric Motor Production Facility project to                       │
│  - Michael Schneider (Project Manager): Oversees project timeline, budget, and cross-functional coordination;   │
│  - Marina Volkova (Automation Engineer): Develops software and control systems for manufacturing automation;    │
│  - Kwame Mensah (Mechanical Designer): Designs motor components and production line equipment;                  │
│  - Emily Carter (QA Engineer): Ensures motors meet aerospace-grade quality and reliability standards;           │
│  - Luca Romano (Manufacturing QA Engineer): Implements inspection processes and production audits;              │
│   based on their skills, availability, and current workload. Ensure that each task is assigned to the most      │
│  suitable team member and that the workload is evenly distributed.  If the same task is allocated to multiple   │
│  team members, split the task into logical subtasks with unique and clear names and allocate one team member    │
│  at most to each subtask.                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Site selection and acquisition",                                                           │
│        "start_week": 1,                                                                                         │
│        "finish_week": 2,                                                                                        │
│        "estimated_time_days": 10.0,                                                                             │
│        "required_resources": ["Michael Schneider"]                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Utility assessments and contracts",                                                        │
│        "start_week": 2,                                                                                         │
│        "finish_week": 3,                                                                                        │
│        "estimated_time_days": 5.0,                                                                              │
│        "required_resources": ["Michael Schneider"]                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Environmental impact assessments",                                                         │
│        "start_week": 3,                                                                                         │
│        "finish_week": 5,                                                                                        │
│        "estimated_time_days": 10.0,                                                                             │
│        "required_resources": ["Michael Schneider"]                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Begin site preparations including earthworks and foundation checks",                       │
│        "start_week": 5,                                                                                         │
│        "finish_week": 9,                                                                                        │
│        "estimated_time_days": 20.0,                                                                             │
│        "required_resources": ["Michael Schneider"]                                                              │
│      },                                                

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 270bb5b7-31ff-428d-b321-3dae70612467                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 323c78a3-137c-4b3e-8813-64d4f1e2ab49                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Site selection and acquisition",                                                           │
│        "start_week": 1,                                                                                         │
│        "finish_week": 2,                                                                                        │
│        "estimated_time_days": 10.0,                                                                             │
│        "required_resources": ["Michael Schneider"]                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Utility assessments and contracts",                                                        │
│        "start_week": 2,                                                                                         │
│        "finish_week": 3,                                                                                        │
│        "estimated_time_days": 5.0,                                                                              │
│        "required_resources": ["Michael Schneider"]                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Environmental impact assessments",                                                         │
│        "start_week": 3,                                                                                         │
│        "finish_week": 5,                                                                                        │
│        "estimated_time_days": 10.0,                                                                             │
│        "required_resources": ["Michael Schneider"]                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Begin site preparations including earthworks and foundation checks",                       │
│        "start_week": 5,                                                                                         │
│        "finish_week": 9,                                                                                        │
│        "estimated_time_days": 20.0,                                                                             │
│        "required_resources": ["Michael Schneider"]    

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [10]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.model_dump()])
df_usage_metrics

Total costs: $0.0016


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,10396,5302,0,5094,3


## Result

In [11]:
result.pydantic.model_dump()

{'tasks': [{'task_name': 'Site selection and acquisition',
   'start_week': 1,
   'finish_week': 2,
   'estimated_time_days': 10.0,
   'required_resources': ['Michael Schneider']},
  {'task_name': 'Utility assessments and contracts',
   'start_week': 2,
   'finish_week': 3,
   'estimated_time_days': 5.0,
   'required_resources': ['Michael Schneider']},
  {'task_name': 'Environmental impact assessments',
   'start_week': 3,
   'finish_week': 5,
   'estimated_time_days': 10.0,
   'required_resources': ['Michael Schneider']},
  {'task_name': 'Begin site preparations including earthworks and foundation checks',
   'start_week': 5,
   'finish_week': 9,
   'estimated_time_days': 20.0,
   'required_resources': ['Michael Schneider']},
  {'task_name': 'Develop initial layout drafts',
   'start_week': 10,
   'finish_week': 12,
   'estimated_time_days': 15.0,
   'required_resources': ['Kwame Mensah']},
  {'task_name': 'Review and finalize production layout with input from all team leads',
   'sta

## Inspect further

In [12]:
tasks = result.pydantic.model_dump()['tasks']
df_tasks = pd.DataFrame(tasks)
df_tasks['estimated_time_days'] = round(df_tasks['estimated_time_days']).astype(int)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,start_week,finish_week,estimated_time_days,required_resources
0,Site selection and acquisition,1,2,10,['Michael Schneider']
1,Utility assessments and contracts,2,3,5,['Michael Schneider']
2,Environmental impact assessments,3,5,10,['Michael Schneider']
3,Begin site preparations including earthworks and foundation checks,5,9,20,['Michael Schneider']
4,Develop initial layout drafts,10,12,15,['Kwame Mensah']
5,Review and finalize production layout with input from all team leads,13,14,10,['Kwame Mensah']
6,Submit for approval and modifications based on feedback,15,15,5,['Kwame Mensah']
7,Finalize mechanical designs for workstations,16,19,20,['Kwame Mensah']
8,Research and select vendors for CNC and robotic machinery,11,12,10,['Marina Volkova']
9,Place orders for machinery,13,13,5,['Marina Volkova']


In [ ]:
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

# Explode team members
df_exploded = df_tasks.explode('required_resources').copy()

# Convert weeks to datetime
project_start = datetime(2026, 1, 1)
df_exploded['start'] = df_exploded['start_week'].apply(lambda w: project_start + timedelta(weeks=int(w)-1))
df_exploded['finish'] = df_exploded['finish_week'].apply(lambda w: project_start + timedelta(weeks=int(w)))

# Create string versions of the dates for compatibility
df_exploded['start_str'] = df_exploded['start'].dt.strftime('%Y-%m-%d')
df_exploded['finish_str'] = df_exploded['finish'].dt.strftime('%Y-%m-%d')

# Sort and set task order for y-axis
df_exploded = df_exploded.sort_values('start')
ordered_tasks = df_exploded['task_name'].unique()[::-1]
df_exploded['task_name'] = pd.Categorical(df_exploded['task_name'], categories=ordered_tasks, ordered=True)

# Create timeline with string dates
fig = px.timeline(
    df_exploded,
    x_start='start_str',
    x_end='finish_str',
    y='task_name',
    color='required_resources',
    title='Electric Motor Production Facility – Gantt Chart',
    template='plotly_dark',
    labels={'required_resources': 'Team Member'}
)

# Reverse Y axis task order
fig.update_yaxes(categoryorder='array', categoryarray=ordered_tasks)

# IMPORTANT: set xaxis type to 'date' so Plotly treats strings as dates
fig.update_layout(
    xaxis=dict(type='date'),
    xaxis_title='Date',
    yaxis_title='',
    legend_title='Team Member',
    font=dict(size=12),
    margin=dict(l=200, r=20, t=60, b=40),
    height=600
)

fig.show()
fig.write_html('gantt_chart.html')


### Inspecting Milestones

In [14]:
milestones = result.pydantic.model_dump()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks,deliverables
0,Site preparation completed,"['Site selection and acquisition', 'Utility assessments and contracts', 'Environmental impact assessments', 'Begin site preparations including earthworks and foundation checks']",['Secured site and utility readiness.']
1,Design finalized,"['Develop initial layout drafts', 'Review and finalize production layout with input from all team leads', 'Submit for approval and modifications based on feedback', 'Finalize mechanical designs for workstations']",['Finalized production layout design.']
2,Equipment installation completed,"['Research and select vendors for CNC and robotic machinery', 'Place orders for machinery', 'Begin installation of CNC machines and robotic systems', 'Installation of additional equipment (winding machines, assembly tools)']",['Installed manufacturing equipment.']
3,MES developed,"['Define MES requirements and develop software specifications', 'Collaborate with IT to build tracking software', 'Pilot testing of MES functionalities']",['Completed and functional MES.']
4,Compliance certification achieved,"['Document all processes and compliance measures', 'Conduct internal audits for compliance verification', 'Prepare for third-party compliance audit']",['Compliance certification.']
5,Testing protocols established,"['Research industry standards for testing metrics', 'Develop detailed testing protocols', 'Create test plans and approval processes']",['Documented testing protocols.']
6,Cleanroom protocols in place,"['Assess areas that require cleanliness protocols', 'Develop and document cleanroom and dust-control measures', 'Train staff on cleanroom protocols']",['Cleanroom protocols in place.']
7,Training completed,"['Develop training materials and manuals', 'Schedule and conduct training sessions', 'Assessment of staff competency post-training']",['Trained staff ready for production.']
8,Supply chain established,"['Research and vet suppliers', 'Negotiate contracts and establish supply agreements', 'Initiate orders for initial material stock']",['Reliable supply chain and signed contracts.']
9,Scalability planned,"['Evaluate potential for modular production lines', 'Design plans for future installations (automation upgrades, etc.)', 'Develop a phased roadmap for future capability expansions']",['Scalability roadmap document.']


# References:

- DeepLearning.AI: [Practical Multi AI Agents and Advanced Use Cases with crewAI](https://www.deeplearning.ai/short-courses/practical-multi-ai-agents-and-advanced-use-cases-with-crewai/)